# Project Asha - Image → MANO θ Training Pipeline

This notebook trains a neural network to predict MANO parameters directly from images, bypassing MediaPipe + IK optimization.

**Architectures:**
- ResNet + MLP: Fast baseline (~15-20mm target)
- Transformer: Better accuracy (~12-15mm target, HandFormer-style)

**Dataset:** FreiHAND (32K poses × 4 views = 130K images)

**Setup:** All code is inline. Clone repo, mount Drive, copy dataset, train, save to Drive.


## 1. Setup Environment

In [ ]:
# clone repository from github
import os
import subprocess
from pathlib import Path

repo_url = "https://github.com/jeevankarandikar/asha.git"
repo_dir = "/content/asha"

if not os.path.exists(repo_dir):
    print(f"cloning repository from github...")
    subprocess.run(['git', 'clone', repo_url, repo_dir], check=True)
    print("✓ repository cloned")
else:
    print(f"✓ repository already exists at {repo_dir}")

os.chdir(repo_dir)

# store paths globally
REPO_ROOT = Path(repo_dir)
MANO_ROOT = REPO_ROOT / 'models'
CHECKPOINT_DIR_LOCAL = REPO_ROOT / 'checkpoints'

print(f"\n✓ working directory: {os.getcwd()}")
print(f"✓ repo root: {REPO_ROOT}")
print(f"✓ mano models: {MANO_ROOT}")
print(f"✓ local checkpoints: {CHECKPOINT_DIR_LOCAL}")


✓ Working directory: /content
✓ Repo root: /content
✓ MANO models: /content/models
✓ Checkpoints: /content/checkpoints


In [ ]:
# install dependencies
import subprocess

# colab comes with torch/torchvision, but we ensure cuda version and install missing packages
packages = [
    'torch',  # colab has this but we ensure latest
    'torchvision',  # colab has this but we ensure latest
    'albumentations',  # data augmentation
    'tqdm',  # progress bars
    'tensorboard',  # logging
    'opencv-python',  # image loading (colab has this but we ensure latest)
    'scipy',  # needed for sparse matrices in mano model
    'matplotlib',  # plotting (colab has this but we ensure latest)
    'h5py',  # dataset loading
]

subprocess.run(['pip', 'install'] + packages + ['-q'], check=True)
print("✓ dependencies installed")

# import all required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import json
import numpy as np
import cv2
import pickle
import albumentations as A
from albumentations.pytorch import ToTensorV2
from pathlib import Path
from typing import Dict, List, Tuple, Optional
from datetime import datetime
import torchvision.models as models
import matplotlib.pyplot as plt
import scipy.sparse

print("✓ all imports loaded")


✓ Dependencies installed
✓ All imports loaded


In [ ]:
# Check GPU and system specs (works in both Colab and local)
import torch
import platform
import subprocess
import os

print("=" * 60)
print("SYSTEM SPECIFICATIONS")
print("=" * 60)

# GPU Info - Works in both Colab and local
is_colab = 'COLAB_GPU' in os.environ or 'google.colab' in str(os.environ)

try:
    # Suppress stderr to avoid "command not found" messages
    result = subprocess.run(
        ['nvidia-smi'], 
        capture_output=True, 
        text=True, 
        timeout=5,
        stderr=subprocess.DEVNULL  # Suppress error messages
    )
    if result.returncode == 0 and result.stdout:
        env_name = 'Colab' if is_colab else 'Local'
        print(f'\n✅ GPU Connected ({env_name}):')
        print(result.stdout)
    else:
        raise FileNotFoundError("nvidia-smi not available")
except (FileNotFoundError, subprocess.TimeoutExpired):
    print('\n⚠️  No GPU detected (running on CPU)')
    print('  Training will be very slow!')
    if not is_colab:
        print('  💡 Tip: Use Google Colab for free GPU access')
except Exception as e:
    print(f'\n⚠️  Could not check GPU status: {e}')
    print('  Assuming CPU mode')

# PyTorch GPU check
print(f"\n🔧 PyTorch:")
print(f"  CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"  Device: {torch.cuda.get_device_name(0)}")
    print(f"  CUDA Version: {torch.version.cuda}")
    print(f"  GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("  ⚠️  Running on CPU - training will be very slow!")
    print("  💡 Tip: Use Google Colab for free GPU access")

# System Info
print(f"\n💻 System:")
print(f"  Platform: {platform.system()}")
print(f"  Python: {platform.python_version()}")
print(f"  PyTorch: {torch.__version__}")

try:
    import multiprocessing
    print(f"  CPU Cores: {multiprocessing.cpu_count()}")
except:
    pass

# Environment info
if is_colab:
    print(f"\n🌐 Environment: Google Colab")
else:
    print(f"\n🌐 Environment: Local ({'CPU' if not torch.cuda.is_available() else 'GPU'})")

print("=" * 60)


SYSTEM SPECIFICATIONS

⚠️  No GPU detected (running on CPU)
  Training will be very slow!

🔧 PyTorch:
  CUDA Available: False
  ⚠️  Running on CPU - training will be very slow!
  💡 Tip: Use Google Colab for free GPU access

💻 System:
  Platform: Linux
  Python: 3.12.12
  PyTorch: 2.9.0+cu126
  CPU Cores: 2

🌐 Environment: Google Colab


## 2. Mount Google Drive & Copy Dataset

**Steps:**
1. Mount Google Drive (cell below)
2. Copy dataset from Drive to Colab
3. Dataset should be at: `MyDrive/datasets/freihand/` or `MyDrive/freihand/`

**If you haven't uploaded yet:**
- Upload your `freihand` folder from `~/datasets/freihand` to Google Drive
- Place it in `MyDrive/datasets/freihand/` or `MyDrive/freihand/`


In [ ]:
# mount google drive and extract dataset zip
from google.colab import drive
import zipfile
from pathlib import Path

print("mounting google drive...")
drive.mount('/content/drive')
print("✓ google drive mounted")

# find zip file in drive
drive_zip_paths = [
    Path('/content/drive/MyDrive/datasets/freihand.zip'),
    Path('/content/drive/MyDrive/freihand.zip'),
]

drive_zip_path = None
for path in drive_zip_paths:
    if path.exists():
        drive_zip_path = path
        print(f"✓ dataset zip found: {path}")
        break

if not drive_zip_path:
    print("\n❌ dataset zip not found in drive")
    print("   upload freihand.zip to mydrive/datasets/")
    raise FileNotFoundError("dataset zip not found in google drive")

# extract zip file to colab
local_dataset_path = Path('/content/freihand')
if local_dataset_path.exists():
    print(f"\n⚠️  dataset already exists at {local_dataset_path}")
    print("   delete it first if you want to re-extract")
else:
    print(f"\nextracting zip file to colab...")
    print(f"   this may take 5-10 minutes for ~3.7gb...")
    with zipfile.ZipFile(drive_zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/')
    print("✓ dataset extracted")

# verify dataset
required_files = ["training_mano.json", "training_xyz.json", "training/rgb"]
missing = [f for f in required_files if not Path(local_dataset_path, f).exists()]

if missing:
    print(f"\n❌ missing files: {missing}")
    raise FileNotFoundError("dataset incomplete")
else:
    print(f"\n✓ dataset verified")
    num_images = len(list((local_dataset_path / "training" / "rgb").glob("*.jpg")))
    print(f"✓ found {num_images:,} images")


## 3. Training Configuration

Configure your training parameters here.


In [ ]:
# training configuration
import os
from pathlib import Path

# dataset path (should be /content/freihand after copying from drive)
dataset_path = '/content/freihand'
if not os.path.exists(dataset_path):
    print(f"⚠️  dataset not found at {dataset_path}")
    print("   make sure you ran the drive mount cell above")
else:
    print(f"✓ dataset path: {dataset_path}")

# setup checkpoint directories
CHECKPOINT_DIR_DRIVE = Path('/content/drive/MyDrive/asha_checkpoints')
CHECKPOINT_DIR_LOCAL = Path('/content/asha/checkpoints')

# Create directories
CHECKPOINT_DIR_DRIVE.mkdir(parents=True, exist_ok=True)
CHECKPOINT_DIR_LOCAL.mkdir(parents=True, exist_ok=True)

print(f"✓ Checkpoints will be saved to:")
print(f"  - Google Drive: {CHECKPOINT_DIR_DRIVE}")
print(f"  - Local backup: {CHECKPOINT_DIR_LOCAL}")

config = {
    # dataset
    'dataset_path': dataset_path,
    'batch_size': 64,  # a100 can handle 64-128, reduce if oom
    'num_workers': 8,  # more workers for a100
    'image_size': 224,
    'use_all_views': True,  # use all 4 views per pose (130k samples) or false (32k samples)
    'train_split': 0.9,  # 90% train, 10% val
    
    # model
    'architecture': 'resnet',  # 'resnet' or 'transformer'
    'backbone': 'resnet50',  # 'resnet50' or 'resnet101'
    'pretrained': True,
    'predict_beta': True,
    
    # training
    'epochs': 50,
    'lr': 1e-4,
    'weight_decay': 1e-5,
    'lr_schedule': 'cosine',  # 'cosine', 'step', or 'none'
    'use_mixed_precision': True,  # use fp16 for faster training on a100
    
    # loss
    'use_mano_loss': False,  # true = slower but more accurate (requires mano forward pass)
    'lambda_param': 1.0,
    'lambda_joint': 10.0,
    'lambda_reg': 0.01,
    
    # output
    'output_dir_drive': str(CHECKPOINT_DIR_DRIVE),  # primary: google drive
    'output_dir_local': str(CHECKPOINT_DIR_LOCAL),  # backup: local
    'save_periodic_checkpoints': False,  # set true to save every 10 epochs (adds ~1-2gb per experiment)
    
    # system
    'gpu': 0,
}

print("\nconfiguration:")
for key, value in config.items():
    print(f"  {key}: {value}")


## 4. Define All Training Code (Inline)

All code is included here - dataset loader, models, losses, and MANO layer. No external files needed!


## 5. Define Models (ResNet & Transformer)


In [ ]:
# ============================================================================
# MODEL ARCHITECTURES
# ============================================================================

class ResNetMANO(nn.Module):
    """resnet backbone + mlp head for mano parameter prediction."""
    
    def __init__(self, backbone: str = "resnet50", pretrained: bool = True,
                 predict_beta: bool = True, dropout: float = 0.5):
        super().__init__()
        
        # backbone
        if backbone == "resnet50":
            resnet = models.resnet50(pretrained=pretrained)
            feature_dim = 2048
        elif backbone == "resnet101":
            resnet = models.resnet101(pretrained=pretrained)
            feature_dim = 2048
        else:
            raise ValueError(f"unknown backbone: {backbone}")
        
        # remove final fc layer
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])
        
        # mlp head for mano parameters
        self.predict_beta = predict_beta
        
        # pose parameters θ (45 dof)
        self.theta_head = nn.Sequential(
            nn.Linear(feature_dim, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 45),
        )
        
        # shape parameters β (10 dof, optional)
        if predict_beta:
            self.beta_head = nn.Sequential(
                nn.Linear(feature_dim, 256),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(256, 10),
            )
    
    def forward(self, x):
        # extract features
        features = self.backbone(x)  # [B, 2048, 1, 1]
        features = features.view(features.size(0), -1)  # [B, 2048]
        
        # predict mano parameters
        theta = self.theta_head(features)  # [B, 45]
        
        if self.predict_beta:
            beta = self.beta_head(features)  # [B, 10]
            return theta, beta
        else:
            return theta


class HandFormer(nn.Module):
    """transformer-based architecture inspired by handformer (2024)."""
    
    def __init__(self, backbone: str = "resnet50", pretrained: bool = True,
                 d_model: int = 256, nhead: int = 8, num_layers: int = 4,
                 dim_feedforward: int = 1024, dropout: float = 0.1, predict_beta: bool = True):
        super().__init__()
        
        # backbone
        if backbone == "resnet50":
            resnet = models.resnet50(pretrained=pretrained)
            feature_dim = 2048
        else:
            raise ValueError(f"unknown backbone: {backbone}")
        
        # remove final layers, keep up to avgpool
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])
        
        # feature projection
        self.feature_proj = nn.Conv2d(feature_dim, d_model, kernel_size=1)
        
        # positional encoding (learned)
        self.pos_encoding = nn.Parameter(torch.randn(1, 196, d_model))  # 14x14 = 196
        
        # transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward,
            dropout=dropout, batch_first=True,
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # global pooling
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        
        # mlp heads
        self.predict_beta = predict_beta
        
        self.theta_head = nn.Sequential(
            nn.Linear(d_model, 512),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(256, 45),
        )
        
        if predict_beta:
            self.beta_head = nn.Sequential(
                nn.Linear(d_model, 256),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(256, 10),
            )
    
    def forward(self, x):
        B = x.size(0)
        
        # extract features
        features = self.backbone(x)  # [B, 2048, H', W']
        features = self.feature_proj(features)  # [B, d_model, H', W']
        
        # flatten spatial dimensions
        B, C, H, W = features.shape
        features = features.view(B, C, H * W).permute(0, 2, 1)  # [B, H*W, d_model]
        
        # add positional encoding
        features = features + self.pos_encoding
        
        # transformer encoding
        encoded = self.transformer(features)  # [B, H*W, d_model]
        
        # global pooling
        pooled = self.global_pool(encoded.permute(0, 2, 1)).squeeze(-1)  # [B, d_model]
        
        # predict mano parameters
        theta = self.theta_head(pooled)  # [B, 45]
        
        if self.predict_beta:
            beta = self.beta_head(pooled)  # [B, 10]
            return theta, beta
        else:
            return theta


def create_model(architecture: str = "resnet", **kwargs) -> nn.Module:
    """factory function to create model."""
    if architecture == "resnet":
        return ResNetMANO(**kwargs)
    elif architecture == "transformer":
        return HandFormer(**kwargs)
    else:
        raise ValueError(f"Unknown architecture: {architecture}")

print("✓ Models defined")


## 6. Define Loss Functions

In [ ]:
# ============================================================================
# LOSS FUNCTIONS
# ============================================================================

class SimpleMANOLoss(nn.Module):
    """Simplified loss: just parameter L2 + regularization. Fast, no MANO forward pass."""
    
    def __init__(self, lambda_param: float = 1.0, lambda_reg: float = 0.01):
        super().__init__()
        self.lambda_param = lambda_param
        self.lambda_reg = lambda_reg
        self.mse_loss = nn.MSELoss()
    
    def forward(self, pred_theta: torch.Tensor, pred_beta: torch.Tensor,
                gt_theta: torch.Tensor, gt_beta: torch.Tensor) -> Dict[str, torch.Tensor]:
        # Parameter loss
        loss_param_theta = self.mse_loss(pred_theta, gt_theta)
        loss_param_beta = self.mse_loss(pred_beta, gt_beta)
        loss_param = loss_param_theta + 0.1 * loss_param_beta
        
        # Regularization
        loss_reg = (pred_theta ** 2).mean() + (pred_beta ** 2).mean()
        
        # Total
        total_loss = self.lambda_param * loss_param + self.lambda_reg * loss_reg
        
        return {
            'param': loss_param,
            'reg': loss_reg,
            'total': total_loss,
        }


class MANOLoss(nn.Module):
    """Multi-term loss: parameter + joint position + regularization. Slower but more accurate."""
    
    def __init__(self, lambda_param: float = 1.0, lambda_joint: float = 10.0,
                 lambda_vertex: float = 1.0, lambda_reg: float = 0.01,
                 use_vertex_loss: bool = False, mano_layer: Optional[object] = None):
        super().__init__()
        self.lambda_param = lambda_param
        self.lambda_joint = lambda_joint
        self.lambda_vertex = lambda_vertex
        self.lambda_reg = lambda_reg
        self.use_vertex_loss = use_vertex_loss
        self.mano_layer = mano_layer
        self.mse_loss = nn.MSELoss()
    
    def forward(self, pred_theta: torch.Tensor, pred_beta: torch.Tensor,
                gt_theta: torch.Tensor, gt_beta: torch.Tensor,
                gt_joints: Optional[torch.Tensor] = None,
                gt_vertices: Optional[torch.Tensor] = None) -> Dict[str, torch.Tensor]:
        losses = {}
        
        # 1. Parameter loss
        loss_param_theta = self.mse_loss(pred_theta, gt_theta)
        loss_param_beta = self.mse_loss(pred_beta, gt_beta)
        loss_param = loss_param_theta + 0.1 * loss_param_beta
        losses['param'] = loss_param
        
        # 2. Joint position loss (if MANO layer available)
        if gt_joints is not None and self.mano_layer is not None:
            pred_verts, pred_joints = self.mano_layer(pred_theta, pred_beta)
            loss_joint = self.mse_loss(pred_joints, gt_joints)
            losses['joint'] = loss_joint
        else:
            loss_joint = torch.tensor(0.0, device=pred_theta.device)
            losses['joint'] = loss_joint
        
        # 3. Vertex loss (optional)
        if self.use_vertex_loss and gt_vertices is not None and self.mano_layer is not None:
            if 'pred_verts' not in locals():
                pred_verts, _ = self.mano_layer(pred_theta, pred_beta)
            loss_vertex = self.mse_loss(pred_verts, gt_vertices)
            losses['vertex'] = loss_vertex
        else:
            loss_vertex = torch.tensor(0.0, device=pred_theta.device)
            losses['vertex'] = loss_vertex
        
        # 4. Regularization
        loss_reg_theta = (pred_theta ** 2).mean()
        loss_reg_beta = (pred_beta ** 2).mean()
        loss_reg = loss_reg_theta + loss_reg_beta
        losses['reg'] = loss_reg
        
        # Total loss
        total_loss = (
            self.lambda_param * loss_param +
            self.lambda_joint * loss_joint +
            self.lambda_vertex * loss_vertex +
            self.lambda_reg * loss_reg
        )
        losses['total'] = total_loss
        
        return losses

print("✓ Loss functions defined")


## 7. Define Dataset Loader (FreiHAND)


In [ ]:
# ============================================================================
# DATASET LOADER (FreiHAND)
# ============================================================================

class FreiHANDDataset(Dataset):
    """FreiHAND dataset loader - 32K poses × 4 views = 130K images."""
    
    def __init__(self, dataset_path: str, split: str = "train", use_augmentation: bool = True,
                 image_size: int = 224, use_all_views: bool = True):
        self.root = Path(dataset_path)
        self.split = split
        self.image_size = image_size
        self.use_all_views = use_all_views
        
        # Load annotations
        mano_path = self.root / "training_mano.json"
        xyz_path = self.root / "training_xyz.json"
        
        if not mano_path.exists():
            raise FileNotFoundError(f"MANO annotations not found: {mano_path}")
        
        with open(mano_path, 'r') as f:
            mano_data = json.load(f)
        with open(xyz_path, 'r') as f:
            xyz_data = json.load(f)
        
        # Extract MANO parameters
        self.mano_params = []
        self.joints_3d = np.array(xyz_data)  # [32560, 21, 3]
        
        for i, mano_dict in enumerate(mano_data):
            params = {
                'theta': np.array(mano_dict['hand_pose'], dtype=np.float32),  # [45]
                'beta': np.array(mano_dict['betas'], dtype=np.float32),  # [10]
                'trans': np.array(mano_dict.get('hand_trans', [0, 0, 0]), dtype=np.float32),
            }
            self.mano_params.append(params)
        
        # Image paths
        self.rgb_dir = self.root / "training" / "rgb"
        
        # Create sample indices
        self.samples = []
        num_unique = len(self.mano_params)  # 32560
        
        if use_all_views:
            # Use all 4 views per pose (130K samples)
            for base_idx in range(num_unique):
                for view_idx in range(4):
                    img_idx = base_idx * 4 + view_idx
                    self.samples.append({
                        'base_idx': base_idx,
                        'view_idx': view_idx,
                        'img_idx': img_idx,
                    })
        else:
            # Use only first view (32K samples)
            for base_idx in range(num_unique):
                self.samples.append({
                    'base_idx': base_idx,
                    'view_idx': 0,
                    'img_idx': base_idx * 4,
                })
        
        print(f"[FreiHAND] Loaded {len(self.samples)} samples")
        print(f"  Unique poses: {num_unique}")
        print(f"  Views per pose: {4 if use_all_views else 1}")
        
        # Data augmentation
        if use_augmentation and split == "train":
            self.transform = A.Compose([
                A.Resize(image_size, image_size),
                A.HorizontalFlip(p=0.5),
                A.Rotate(limit=15, p=0.5),
                A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1, p=0.5),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ])
        else:
            self.transform = A.Compose([
                A.Resize(image_size, image_size),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ])
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        base_idx = sample['base_idx']
        img_idx = sample['img_idx']
        
        # Load image
        img_path = self.rgb_dir / f"{img_idx:08d}.jpg"
        image = cv2.imread(str(img_path))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Apply transforms
        transformed = self.transform(image=image)
        image_tensor = transformed['image']  # [3, H, W]
        
        # Get MANO parameters
        mano = self.mano_params[base_idx]
        theta = torch.from_numpy(mano['theta'])  # [45]
        beta = torch.from_numpy(mano['beta'])  # [10]
        
        # Get 3D joints
        joints_3d = torch.from_numpy(self.joints_3d[base_idx])  # [21, 3]
        
        return {
            'image': image_tensor,
            'theta': theta,
            'beta': beta,
            'joints_3d': joints_3d,
            'base_idx': base_idx,
            'view_idx': sample['view_idx'],
        }


def create_dataloaders(freihand_path: str, ho3d_path: Optional[str] = None,
                       batch_size: int = 32, num_workers: int = 4, image_size: int = 224,
                       use_all_views: bool = True, train_split: float = 0.9):
    """
    Create train and validation dataloaders.
    
    Args:
        freihand_path: path to FreiHAND dataset
        ho3d_path: optional path to HO-3D dataset (not implemented yet)
        batch_size: batch size for training
        num_workers: number of data loading workers
        image_size: target image size
        use_all_views: use all 4 views per FreiHAND pose (130K samples) or False (32K samples)
        train_split: fraction of data for training (default 0.9 = 90% train, 10% val)
    
    Returns:
        train_loader, val_loader
    """
    # Create FreiHAND dataset
    freihand_full = FreiHANDDataset(
        dataset_path=freihand_path,
        split="train",
        use_augmentation=True,
        image_size=image_size,
        use_all_views=use_all_views,
    )
    
    # Split into train/val (90% train, 10% val by default)
    total_size = len(freihand_full)
    train_size = int(train_split * total_size)
    val_size = total_size - train_size
    
    train_dataset, val_dataset = torch.utils.data.random_split(
        freihand_full,
        [train_size, val_size],
        generator=torch.Generator().manual_seed(42)  # Reproducible split
    )
    
    # HO-3D integration (placeholder - not implemented yet)
    if ho3d_path:
        print("⚠️  HO-3D dataset support not yet implemented")
        print("   Set ho3d_path=None to use only FreiHAND")
        # TODO: Implement HO3DDataset and combine
    
    # Create dataloaders with GPU optimizations
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        prefetch_factor=2 if torch.cuda.is_available() else None,
        persistent_workers=True if num_workers > 0 and torch.cuda.is_available() else False,
        drop_last=True,
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True if torch.cuda.is_available() else False,
        prefetch_factor=2 if torch.cuda.is_available() else None,
        persistent_workers=True if num_workers > 0 and torch.cuda.is_available() else False,
    )
    
    print(f"\n[DataLoaders]")
    print(f"  Train: {len(train_dataset)} samples ({train_split*100:.0f}%)")
    print(f"  Val: {len(val_dataset)} samples ({(1-train_split)*100:.0f}%)")
    print(f"  Batch size: {batch_size}")
    
    return train_loader, val_loader

print("✓ Dataset loader defined")


In [ ]:
# ============================================================================
# MANO LAYER (for forward pass: parameters → 3D mesh)
# ============================================================================

class CustomMANOLayer:
    """Custom MANO model loader - converts parameters to 3D hand mesh."""
    
    def __init__(self, mano_root: str, side: str = "right", device: str = "cpu"):
        self.mano_root = Path(mano_root)
        self.side = side.upper()
        self.device = device
        self.data = self._load_mano_model()
        self._setup_tensors()
    
    def _load_mano_model(self):
        """Load MANO model from converted numpy pkl file."""
        pkl_path = self.mano_root / f"MANO_{self.side}_numpy.pkl"
        if not pkl_path.exists():
            raise FileNotFoundError(f"MANO model not found: {pkl_path}")
        with open(pkl_path, 'rb') as f:
            return pickle.load(f)
    
    def _setup_tensors(self):
        """Convert numpy arrays to PyTorch tensors."""
        self.shapedirs = torch.from_numpy(self.data['shapedirs'].astype(np.float32)).to(self.device)
        self.posedirs = torch.from_numpy(self.data['posedirs'].astype(np.float32)).to(self.device)
        self.v_template = torch.from_numpy(self.data['v_template'].astype(np.float32)).to(self.device)
        self.J_regressor = torch.from_numpy(self.data['J_regressor'].toarray().astype(np.float32)).to(self.device)
        self.weights = torch.from_numpy(self.data['weights'].astype(np.float32)).to(self.device)
        self.kintree_table = self.data['kintree_table']
        self.faces = torch.from_numpy(self.data['f'].astype(np.int64)).to(self.device)
        self.fingertip_indices = torch.tensor([745, 317, 444, 556, 673], dtype=torch.long, device=self.device)
    
    def rodrigues(self, theta: torch.Tensor) -> torch.Tensor:
        """Convert axis-angle to rotation matrix using Rodrigues formula."""
        batch_size = theta.shape[0]
        device = theta.device
        angle = torch.norm(theta + 1e-8, dim=1, keepdim=True)
        r = theta / angle
        rx, ry, rz = r[:, 0], r[:, 1], r[:, 2]
        K = torch.zeros((batch_size, 3, 3), device=device)
        K[:, 0, 1] = -rz; K[:, 0, 2] = ry
        K[:, 1, 0] = rz; K[:, 1, 2] = -rx
        K[:, 2, 0] = -ry; K[:, 2, 1] = rx
        K_squared = torch.bmm(K, K)
        angle = angle.unsqueeze(-1)
        I = torch.eye(3, device=device).unsqueeze(0).repeat(batch_size, 1, 1)
        R = I + torch.sin(angle) * K + (1 - torch.cos(angle)) * K_squared
        return R
    
    def batch_rodrigues(self, theta: torch.Tensor) -> torch.Tensor:
        """Apply Rodrigues to multiple axis-angle vectors."""
        batch_size = theta.shape[0]
        if theta.dim() == 2:
            theta = theta.view(batch_size, -1, 3)
        num_joints = theta.shape[1]
        theta_flat = theta.view(-1, 3)
        R_flat = self.rodrigues(theta_flat)
        return R_flat.view(batch_size, num_joints, 3, 3)
    
    def with_zeros(self, x: torch.Tensor) -> torch.Tensor:
        """Append homogeneous row [0,0,0,1] to 3x4 matrix."""
        batch_size = x.shape[0]
        bottom = torch.tensor([0.0, 0.0, 0.0, 1.0], device=x.device).unsqueeze(0).unsqueeze(0)
        bottom = bottom.repeat(batch_size, 1, 1)
        return torch.cat([x, bottom], dim=1)
    
    def pack(self, R: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
        """Pack rotation and translation into 4x4 homogeneous matrix."""
        transform_3x4 = torch.cat([R, t], dim=2)
        return self.with_zeros(transform_3x4)
    
    def forward(self, pose: torch.Tensor, betas: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        """Forward pass: parameters → 3D hand mesh."""
        batch_size = pose.shape[0]
        device = pose.device
        
        # Step 1: Shape blend shapes
        v_shaped = self.v_template + torch.einsum('vij,bj->bvi', self.shapedirs, betas)
        
        # Step 2: Rest pose joints
        J = torch.einsum('jv,bvi->bji', self.J_regressor, v_shaped)  # [B, 16, 3]
        
        # Step 3: Pose rotations
        pose_cube = pose.view(batch_size, -1, 3)  # [B, 15, 3]
        rot_mats = self.batch_rodrigues(pose_cube)  # [B, 15, 3, 3]
        
        # Step 4: Pose blend shapes
        I_cube = torch.eye(3, device=device).unsqueeze(0).unsqueeze(0).repeat(batch_size, rot_mats.shape[1], 1, 1)
        pose_feature = (rot_mats - I_cube).view(batch_size, -1)  # [B, 135]
        
        if self.posedirs.dim() == 3:
            v_posed = v_shaped + torch.einsum('vij,bj->bvi', self.posedirs, pose_feature)
        else:
            v_posed = v_shaped + torch.einsum('ij,bj->bi', self.posedirs, pose_feature).view(batch_size, -1, 3)
        
        # Step 5: Forward kinematics
        num_joints = J.shape[1]
        G_list = []
        for i in range(num_joints):
            parent_idx = int(self.kintree_table[0, i]) if i > 0 else -1
            if i == 0 or parent_idx < 0:
                R_i = rot_mats[:, i, :, :] if i < rot_mats.shape[1] else torch.eye(3, device=device).unsqueeze(0).repeat(batch_size, 1, 1)
                t_i = J[:, i, :].unsqueeze(-1)
                G_i = self.pack(R_i, t_i)
                G_list.append(G_i)
            else:
                R_i = rot_mats[:, i, :, :] if i < rot_mats.shape[1] else torch.eye(3, device=device).unsqueeze(0).repeat(batch_size, 1, 1)
                t_rel = (J[:, i, :] - J[:, parent_idx, :]).unsqueeze(-1)
                T_local = self.pack(R_i, t_rel)
                G_i = torch.matmul(G_list[parent_idx], T_local)
                G_list.append(G_i)
        
        G = torch.stack(G_list, dim=1)  # [B, 16, 4, 4]
        
        # Skinning transforms
        G_skinning_list = []
        for i in range(num_joints):
            J_rest_inv = -J[:, i, :].unsqueeze(-1)
            I_batch = torch.eye(3, device=device).unsqueeze(0).repeat(batch_size, 1, 1)
            T_rest_inv = self.pack(I_batch, J_rest_inv)
            G_skin_i = torch.matmul(G[:, i, :, :].clone(), T_rest_inv)
            G_skinning_list.append(G_skin_i)
        G_skinning = torch.stack(G_skinning_list, dim=1)  # [B, 16, 4, 4]
        
        # Step 6: Linear blend skinning
        v_posed_homo = torch.cat([v_posed, torch.ones((batch_size, v_posed.shape[1], 1), device=device)], dim=2)
        W = self.weights.unsqueeze(0).repeat(batch_size, 1, 1)  # [B, 778, 16]
        T = torch.einsum('bjkl,bvl->bvjk', G_skinning, v_posed_homo)  # [B, 778, 16, 4]
        v_final_homo = torch.einsum('bvj,bvjk->bvk', W, T)  # [B, 778, 4]
        vertices = v_final_homo[:, :, :3]
        
        # Joint positions
        joints_posed = G[:, :, :3, 3]  # [B, 16, 3]
        J_fingertips = vertices[:, self.fingertip_indices, :]  # [B, 5, 3]
        joints_final = torch.cat([joints_posed, J_fingertips], dim=1)  # [B, 21, 3]
        
        return vertices, joints_final
    
    def __call__(self, pose: torch.Tensor, betas: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.forward(pose, betas)

print("✓ MANO layer defined")


## 8. Create DataLoaders

In [ ]:
# create dataloaders
print(f"\n[Creating DataLoaders]")
print(f"  dataset: {config['dataset_path']}")
print(f"  batch size: {config['batch_size']}")
print(f"  num workers: {config['num_workers']}")

train_loader, val_loader = create_dataloaders(
    freihand_path=config['dataset_path'],
    ho3d_path=None,  # not using ho3d for now
    batch_size=config['batch_size'],
    num_workers=config['num_workers'],
    image_size=config['image_size'],
    use_all_views=config['use_all_views'],
    train_split=config['train_split'],
)

print(f"\n✓ dataloaders created:")
print(f"  train batches: {len(train_loader)}")
print(f"  val batches: {len(val_loader)}")


## 9. Create Model

In [ ]:
# Device
device = torch.device(f"cuda:{config['gpu']}" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Create model
print(f"\n[Creating model: {config['architecture']}]")
model = create_model(
    architecture=config['architecture'],
    backbone=config['backbone'],
    pretrained=config['pretrained'],
    predict_beta=config['predict_beta'],
)
model = model.to(device)

# Count parameters
num_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"  Total parameters: {num_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")


## 10. Setup Loss, Optimizer, and Scheduler

In [ ]:
# Load MANO layer (for MANO-aware loss)
mano_layer = None
if config['use_mano_loss']:
    print("[Loading MANO layer for loss computation]")
    mano_root = MANO_ROOT if 'MANO_ROOT' in globals() else Path(REPO_ROOT / 'models')
    mano_layer = CustomMANOLayer(
        mano_root=str(mano_root),
        side="right",
        device=str(device)
    )
    print("✓ MANO layer loaded")
else:
    print("Using simple parameter loss (faster)")

# Loss function
if config['use_mano_loss']:
    criterion = MANOLoss(
        lambda_param=config['lambda_param'],
        lambda_joint=config['lambda_joint'],
        lambda_reg=config['lambda_reg'],
        mano_layer=mano_layer,
    )
else:
    criterion = SimpleMANOLoss(
        lambda_param=config['lambda_param'],
        lambda_reg=config['lambda_reg'],
    )

# Optimizer
optimizer = optim.AdamW(
    model.parameters(),
    lr=config['lr'],
    weight_decay=config['weight_decay'],
)

# Learning rate scheduler
if config['lr_schedule'] == "cosine":
    scheduler = optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=config['epochs']
    )
elif config['lr_schedule'] == "step":
    scheduler = optim.lr_scheduler.StepLR(
        optimizer, step_size=config['epochs'] // 3, gamma=0.1
    )
else:
    scheduler = None

# Mixed precision scaler (for FP16 training on A100)
scaler = None
if config['use_mixed_precision'] and torch.cuda.is_available():
    scaler = torch.cuda.amp.GradScaler()
    print("✓ Mixed precision (FP16) enabled for faster training")

print("✓ Loss, optimizer, and scheduler configured")


## 11. Setup Logging and Checkpointing


In [ ]:
# Create output directory structure (save to both Drive and local)
import shutil

output_dir_drive = Path(config['output_dir_drive'])
output_dir_local = Path(config['output_dir_local'])
output_dir_drive.mkdir(parents=True, exist_ok=True)
output_dir_local.mkdir(parents=True, exist_ok=True)

# Create experiment name with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
experiment_name = f"{config['architecture']}_{config['backbone']}_{timestamp}"

# Create experiment directories
experiment_dir_drive = output_dir_drive / experiment_name
experiment_dir_local = output_dir_local / experiment_name
experiment_dir_drive.mkdir(parents=True, exist_ok=True)
experiment_dir_local.mkdir(parents=True, exist_ok=True)

# Use Drive as primary
experiment_dir = experiment_dir_drive

# TensorBoard (save to Drive)
log_dir = experiment_dir / "logs"
writer = SummaryWriter(log_dir=str(log_dir))
print(f"✓ TensorBoard logs: {log_dir}")

# Save config (to both locations)
config_path_drive = experiment_dir_drive / "config.json"
config_path_local = experiment_dir_local / "config.json"
with open(config_path_drive, 'w') as f:
    json.dump(config, f, indent=2)
shutil.copy(config_path_drive, config_path_local)
print(f"✓ Config saved to Drive and local")

print(f"✓ Checkpoints will be saved to:")
print(f"  - Primary (Google Drive): {experiment_dir_drive}")
print(f"  - Backup (Local): {experiment_dir_local}")

# Estimate storage requirements
print(f"\nStorage estimate:")
print(f"  - best.pth: ~300MB (ResNet) / ~500MB (Transformer)")
print(f"  - latest.pth: ~300MB (ResNet) / ~500MB (Transformer)")
if config['save_periodic_checkpoints']:
    num_periodic = config['epochs'] // 10
    print(f"  - Periodic checkpoints ({num_periodic} files): ~{num_periodic * 300}MB (ResNet) / ~{num_periodic * 500}MB (Transformer)")
    print(f"  - Total per experiment: ~{2 + num_periodic}GB (ResNet) / ~{3 + num_periodic * 0.5}GB (Transformer)")
else:
    print(f"  - Periodic checkpoints: Disabled (saves ~1-2GB)")
    print(f"  - Total per experiment: ~600MB-1GB")
print(f"\nNote: Checkpoints are gitignored. Clean up old experiments manually if needed.")


## 12. Training Functions


In [ ]:
def train_epoch(model, dataloader, criterion, optimizer, device, epoch, use_mano_loss=False, mano_layer=None, scaler=None, use_mixed_precision=False):
    """Train for one epoch with optional mixed precision."""
    model.train()
    total_loss = 0.0
    loss_components = {}
    
    pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
    for batch_idx, batch in enumerate(pbar):
        images = batch['image'].to(device, non_blocking=True)
        gt_theta = batch['theta'].to(device, non_blocking=True)
        gt_beta = batch['beta'].to(device, non_blocking=True)
        
        # Mixed precision forward pass
        if use_mixed_precision and scaler is not None:
            with torch.cuda.amp.autocast():
                # Forward pass
                if hasattr(model, 'predict_beta') and model.predict_beta:
                    pred_theta, pred_beta = model(images)
                else:
                    pred_theta = model(images)
                    pred_beta = torch.zeros_like(gt_beta)
                
                # Compute loss
                if use_mano_loss and mano_layer is not None:
                    gt_joints = batch['joints_3d'].to(device, non_blocking=True)
                    losses = criterion(pred_theta, pred_beta, gt_theta, gt_beta, gt_joints=gt_joints)
                else:
                    losses = criterion(pred_theta, pred_beta, gt_theta, gt_beta)
                
                loss = losses['total']
            
            # Backward pass with mixed precision
            optimizer.zero_grad()
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            # Standard precision forward pass
            if hasattr(model, 'predict_beta') and model.predict_beta:
                pred_theta, pred_beta = model(images)
            else:
                pred_theta = model(images)
                pred_beta = torch.zeros_like(gt_beta)
            
            # Compute loss
            if use_mano_loss and mano_layer is not None:
                gt_joints = batch['joints_3d'].to(device, non_blocking=True)
                losses = criterion(pred_theta, pred_beta, gt_theta, gt_beta, gt_joints=gt_joints)
            else:
                losses = criterion(pred_theta, pred_beta, gt_theta, gt_beta)
            
            loss = losses['total']
            
            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        # Accumulate losses
        total_loss += loss.item()
        for key, value in losses.items():
            if key not in loss_components:
                loss_components[key] = 0.0
            loss_components[key] += value.item()
        
        # Update progress bar
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})
    
    # Average losses
    num_batches = len(dataloader)
    avg_loss = total_loss / num_batches
    for key in loss_components:
        loss_components[key] /= num_batches
    loss_components['total'] = avg_loss
    
    return loss_components


def validate(model, dataloader, criterion, device, use_mano_loss=False, mano_layer=None):
    """Validate model."""
    model.eval()
    total_loss = 0.0
    loss_components = {}
    
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Validation"):
            images = batch['image'].to(device)
            gt_theta = batch['theta'].to(device)
            gt_beta = batch['beta'].to(device)
            
            # Forward pass
            if hasattr(model, 'predict_beta') and model.predict_beta:
                pred_theta, pred_beta = model(images)
            else:
                pred_theta = model(images)
                pred_beta = torch.zeros_like(gt_beta)
            
            # Compute loss
            if use_mano_loss and mano_layer is not None:
                gt_joints = batch['joints_3d'].to(device)
                losses = criterion(pred_theta, pred_beta, gt_theta, gt_beta, gt_joints=gt_joints)
            else:
                losses = criterion(pred_theta, pred_beta, gt_theta, gt_beta)
            
            loss = losses['total']
            total_loss += loss.item()
            
            for key, value in losses.items():
                if key not in loss_components:
                    loss_components[key] = 0.0
                loss_components[key] += value.item()
    
    # Average losses
    num_batches = len(dataloader)
    avg_loss = total_loss / num_batches
    for key in loss_components:
        loss_components[key] /= num_batches
    loss_components['total'] = avg_loss
    
    return loss_components

print("✓ Training functions defined")


## 13. Training Loop


In [ ]:
# Training loop
best_val_loss = float('inf')
history = {
    'train_loss': [],
    'val_loss': [],
    'lr': [],
}

print(f"\n{'='*60}")
print(f"Starting training for {config['epochs']} epochs")
print(f"{'='*60}\n")

for epoch in range(config['epochs']):
    # Train
    train_losses = train_epoch(
        model, train_loader, criterion, optimizer, device, epoch,
        config['use_mano_loss'], mano_layer, scaler, config['use_mixed_precision']
    )
    
    # Validate
    val_losses = validate(
        model, val_loader, criterion, device,
        config['use_mano_loss'], mano_layer
    )
    
    # Update learning rate
    if scheduler:
        scheduler.step()
    
    # Log to TensorBoard
    for key, value in train_losses.items():
        writer.add_scalar(f'Train/{key}', value, epoch)
    for key, value in val_losses.items():
        writer.add_scalar(f'Val/{key}', value, epoch)
    writer.add_scalar('LR', optimizer.param_groups[0]['lr'], epoch)
    
    # Save history
    history['train_loss'].append(train_losses['total'])
    history['val_loss'].append(val_losses['total'])
    history['lr'].append(optimizer.param_groups[0]['lr'])
    
    # Print progress
    print(f"\nEpoch {epoch+1}/{config['epochs']}")
    print(f"  Train loss: {train_losses['total']:.4f}")
    print(f"  Val loss: {val_losses['total']:.4f}")
    print(f"  LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    # Save checkpoint
    is_best = val_losses['total'] < best_val_loss
    if is_best:
        best_val_loss = val_losses['total']
    
    checkpoint = {
        'epoch': epoch + 1,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
        'train_loss': train_losses['total'],
        'val_loss': val_losses['total'],
        'best_val_loss': best_val_loss,
        'config': config,
        'history': history,
    }
    
    # Save latest (to both Drive and local)
    torch.save(checkpoint, experiment_dir_drive / "latest.pth")
    torch.save(checkpoint, experiment_dir_local / "latest.pth")
    
    # Save best (to both Drive and local)
    if is_best:
        torch.save(checkpoint, experiment_dir_drive / "best.pth")
        torch.save(checkpoint, experiment_dir_local / "best.pth")
        print(f"  ✓ Saved best model (val_loss: {best_val_loss:.4f})")
    
    # Save periodic checkpoints (optional, adds ~1-2GB per experiment)
    if config['save_periodic_checkpoints'] and (epoch + 1) % 10 == 0:
        checkpoint_path_drive = experiment_dir_drive / f"checkpoint_epoch_{epoch+1}.pth"
        checkpoint_path_local = experiment_dir_local / f"checkpoint_epoch_{epoch+1}.pth"
        torch.save(checkpoint, checkpoint_path_drive)
        torch.save(checkpoint, checkpoint_path_local)

print(f"\n{'='*60}")
print(f"Training complete!")
print(f"Best validation loss: {best_val_loss:.4f}")
print(f"Checkpoints saved to: {experiment_dir}")
print(f"{'='*60}")


## 14. Training History Visualization


In [ ]:
# Plot training history
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train')
plt.plot(history['val_loss'], label='Val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training History')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history['lr'], label='Learning Rate')
plt.xlabel('Epoch')
plt.ylabel('LR')
plt.title('Learning Rate Schedule')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig(experiment_dir / "training_history.png", dpi=150)
plt.show()

print(f"✓ Training history saved: {experiment_dir / 'training_history.png'}")


## 15. Evaluation

Evaluate the best model on validation set with detailed metrics.


In [ ]:
# Load best model
print("[Loading best checkpoint]")
checkpoint = torch.load(experiment_dir / "best.pth", map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✓ Loaded epoch {checkpoint['epoch']} (val_loss: {checkpoint['val_loss']:.4f})")

# Evaluation function
def compute_mpjpe(pred_joints, gt_joints):
    """Compute Mean Per-Joint Position Error (MPJPE) in mm."""
    error = torch.norm(pred_joints - gt_joints, dim=2)  # [B, 21]
    mpjpe = error.mean() * 1000  # convert to mm
    return mpjpe.item()

# Evaluate
model.eval()
param_errors_theta = []
param_errors_beta = []
joint_errors = []
per_joint_errors = []

# Load MANO layer for evaluation
mano_root = MANO_ROOT if 'MANO_ROOT' in globals() else Path(REPO_ROOT / 'models')
mano_layer_eval = CustomMANOLayer(
    mano_root=str(mano_root),
    side="right",
    device=str(device)
)

print("\n[Evaluating on validation set]")
with torch.no_grad():
    for batch in tqdm(val_loader):
        images = batch['image'].to(device)
        gt_theta = batch['theta'].to(device)
        gt_beta = batch['beta'].to(device)
        gt_joints = batch['joints_3d'].to(device)
        
        # Forward pass
        if hasattr(model, 'predict_beta') and model.predict_beta:
            pred_theta, pred_beta = model(images)
        else:
            pred_theta = model(images)
            pred_beta = torch.zeros_like(gt_beta)
        
        # Parameter errors
        theta_error = torch.norm(pred_theta - gt_theta, dim=1).mean().item()
        beta_error = torch.norm(pred_beta - gt_beta, dim=1).mean().item()
        param_errors_theta.append(theta_error)
        param_errors_beta.append(beta_error)
        
        # Forward through MANO to get joints
        pred_verts, pred_joints = mano_layer_eval(pred_theta, pred_beta)
        
        # Joint position errors
        batch_mpjpe = compute_mpjpe(pred_joints, gt_joints)
        joint_errors.append(batch_mpjpe)
        
        # Per-joint errors
        per_joint_error = torch.norm(pred_joints - gt_joints, dim=2) * 1000  # [B, 21] in mm
        per_joint_errors.append(per_joint_error.cpu().numpy())

# Aggregate results
results = {
    'param_error_theta': {
        'mean': np.mean(param_errors_theta),
        'std': np.std(param_errors_theta),
        'median': np.median(param_errors_theta),
    },
    'param_error_beta': {
        'mean': np.mean(param_errors_beta),
        'std': np.std(param_errors_beta),
        'median': np.median(param_errors_beta),
    },
    'joint_error_mm': {
        'mean': np.mean(joint_errors),
        'std': np.std(joint_errors),
        'median': np.median(joint_errors),
        'p95': np.percentile(joint_errors, 95),
    },
    'per_joint_error_mm': np.concatenate(per_joint_errors, axis=0).mean(axis=0).tolist(),
}

# Print results
print("\n" + "="*60)
print("EVALUATION RESULTS")
print("="*60)
print(f"\nParameter Errors:")
print(f"  θ (pose): {results['param_error_theta']['mean']:.4f} ± {results['param_error_theta']['std']:.4f}")
print(f"  β (shape): {results['param_error_beta']['mean']:.4f} ± {results['param_error_beta']['std']:.4f}")
print(f"\nJoint Position Error (MPJPE):")
print(f"  Mean: {results['joint_error_mm']['mean']:.2f} mm")
print(f"  Median: {results['joint_error_mm']['median']:.2f} mm")
print(f"  95th percentile: {results['joint_error_mm']['p95']:.2f} mm")

# Save results
# Save results (to both Drive and local)
results_path_drive = experiment_dir_drive / "evaluation_results.json"
results_path_local = experiment_dir_local / "evaluation_results.json"

with open(results_path_drive, 'w') as f:
    json.dump(results, f, indent=2)
shutil.copy(results_path_drive, results_path_local)

print(f"\n✓ Results saved to:")
print(f"  - Google Drive: {results_path_drive}")
print(f"  - Local backup: {results_path_local}")


## 16. Evaluation Results Visualization

Plot evaluation metrics: per-joint errors, error distributions, and comparisons.


In [ ]:
# plot evaluation results
import matplotlib.pyplot as plt
import numpy as np

# joint names for mano hand (21 joints)
joint_names = [
    'Wrist', 'Thumb1', 'Thumb2', 'Thumb3', 'Thumb4',
    'Index1', 'Index2', 'Index3', 'Index4',
    'Middle1', 'Middle2', 'Middle3', 'Middle4',
    'Ring1', 'Ring2', 'Ring3', 'Ring4',
    'Pinky1', 'Pinky2', 'Pinky3', 'Pinky4',
]

# create figure with subplots
fig = plt.figure(figsize=(16, 10))

# 1. per-joint error bar chart
ax1 = plt.subplot(2, 3, 1)
per_joint_means = results['per_joint_error_mm']
x_pos = np.arange(len(joint_names))
ax1.bar(x_pos, per_joint_means, alpha=0.7, color='steelblue')
ax1.set_xlabel('Joint')
ax1.set_ylabel('Error (mm)')
ax1.set_title('Per-Joint Position Error')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(joint_names, rotation=45, ha='right', fontsize=8)
ax1.grid(True, alpha=0.3)
ax1.axhline(y=results['joint_error_mm']['mean'], color='r', linestyle='--', label=f"Mean MPJPE: {results['joint_error_mm']['mean']:.2f}mm")
ax1.legend()

# 2. joint error distribution histogram
ax2 = plt.subplot(2, 3, 2)
ax2.hist(joint_errors, bins=50, alpha=0.7, color='steelblue', edgecolor='black')
ax2.axvline(results['joint_error_mm']['mean'], color='r', linestyle='--', label=f"Mean: {results['joint_error_mm']['mean']:.2f}mm")
ax2.axvline(results['joint_error_mm']['median'], color='g', linestyle='--', label=f"Median: {results['joint_error_mm']['median']:.2f}mm")
ax2.set_xlabel('MPJPE (mm)')
ax2.set_ylabel('Frequency')
ax2.set_title('Joint Error Distribution')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. parameter error comparison
ax3 = plt.subplot(2, 3, 3)
param_errors = [
    results['param_error_theta']['mean'],
    results['param_error_beta']['mean'],
]
param_labels = ['θ (pose)', 'β (shape)']
colors = ['steelblue', 'coral']
bars = ax3.bar(param_labels, param_errors, alpha=0.7, color=colors)
ax3.set_ylabel('Error')
ax3.set_title('Parameter Errors')
ax3.grid(True, alpha=0.3, axis='y')
# add value labels on bars
for bar, err in zip(bars, param_errors):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
             f'{err:.4f}', ha='center', va='bottom')

# 4. error statistics comparison
ax4 = plt.subplot(2, 3, 4)
stats = ['Mean', 'Median', '95th %ile']
mpjpe_stats = [
    results['joint_error_mm']['mean'],
    results['joint_error_mm']['median'],
    results['joint_error_mm']['p95'],
]
bars = ax4.bar(stats, mpjpe_stats, alpha=0.7, color='steelblue')
ax4.set_ylabel('MPJPE (mm)')
ax4.set_title('MPJPE Statistics')
ax4.grid(True, alpha=0.3, axis='y')
# add value labels
for bar, val in zip(bars, mpjpe_stats):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.2f}', ha='center', va='bottom')

# 5. per-joint error heatmap (if we have per-joint data)
ax5 = plt.subplot(2, 3, 5)
# reshape per-joint errors for visualization
per_joint_array = np.array(per_joint_means).reshape(-1, 1)
im = ax5.imshow(per_joint_array, cmap='YlOrRd', aspect='auto')
ax5.set_yticks(range(len(joint_names)))
ax5.set_yticklabels(joint_names, fontsize=8)
ax5.set_xticks([0])
ax5.set_xticklabels(['Error (mm)'])
ax5.set_title('Per-Joint Error Heatmap')
plt.colorbar(im, ax=ax5)

# 6. cumulative error distribution
ax6 = plt.subplot(2, 3, 6)
sorted_errors = np.sort(joint_errors)
cumulative = np.arange(1, len(sorted_errors) + 1) / len(sorted_errors)
ax6.plot(sorted_errors, cumulative * 100, linewidth=2, color='steelblue')
ax6.axvline(results['joint_error_mm']['mean'], color='r', linestyle='--', label=f"Mean: {results['joint_error_mm']['mean']:.2f}mm")
ax6.axvline(results['joint_error_mm']['median'], color='g', linestyle='--', label=f"Median: {results['joint_error_mm']['median']:.2f}mm")
ax6.set_xlabel('MPJPE (mm)')
ax6.set_ylabel('Cumulative %')
ax6.set_title('Cumulative Error Distribution')
ax6.grid(True, alpha=0.3)
ax6.legend()

plt.tight_layout()

# save plots
plot_path_drive = experiment_dir_drive / "evaluation_plots.png"
plot_path_local = experiment_dir_local / "evaluation_plots.png"
plt.savefig(plot_path_drive, dpi=150, bbox_inches='tight')
shutil.copy(plot_path_drive, plot_path_local)
plt.show()

print(f"\n✓ evaluation plots saved:")
print(f"  - google drive: {plot_path_drive}")
print(f"  - local backup: {plot_path_local}")


## 17. Access Checkpoints

All checkpoints are automatically saved to Google Drive. Access them from:
- **Google Drive**: `MyDrive/asha_checkpoints/{experiment_name}/`
- **Colab file browser**: `/content/drive/MyDrive/asha_checkpoints/{experiment_name}/`


In [ ]:
# Checkpoint summary
print("=" * 60)
print("CHECKPOINT FILES")
print("=" * 60)
print(f"\n📁 Experiment: {experiment_name}")
print(f"\n💾 Google Drive location:")
print(f"   {experiment_dir_drive}")
print(f"\n📦 Files saved:")
print(f"   ✓ best.pth: Best model checkpoint")
print(f"   ✓ latest.pth: Latest checkpoint")
print(f"   ✓ config.json: Training configuration")
print(f"   ✓ evaluation_results.json: Evaluation metrics")
print(f"   ✓ training_history.png: Training curves")
print(f"\n🔗 Access from:")
print(f"   - Google Drive: MyDrive/asha_checkpoints/{experiment_name}/")
print(f"   - Colab file browser: /content/drive/MyDrive/asha_checkpoints/{experiment_name}/")
print(f"\n💡 Tip: Files are automatically synced to Drive. No manual download needed!")
print("=" * 60)


## 18. TensorBoard

View training metrics in TensorBoard. Run this cell separately.


In [ ]:
# Load TensorBoard extension
%load_ext tensorboard

# Start TensorBoard (uncomment to run)
# %tensorboard --logdir {CHECKPOINT_DIR} --port 6006
